Auto-Clean-Test

In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean

/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digennoise = pickle.load(f)

#print(digennoise.keys())

with open("./data/DigenData/all_new_test_sets_no_noise.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())
#print(digen['digen8_4426'])
Test= pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([Test, yval], axis=1)
out.index.name = 'InstanceId'
out.to_csv('./data/DigenStudy/digen8_4426.csv')
print(out)

                  X0        X1        X2        X3        X4        X5  \
InstanceId                                                               
0           1.764052  0.400157  0.978738  2.240893  1.867558 -0.977278   
1           0.144044  1.454274  0.761038  0.121675  0.443863  0.333674   
2          -2.552990  0.653619  0.864436 -0.742165  2.269755 -1.454366   
3           0.154947  0.378163 -0.887786 -1.980796 -0.347912  0.156349   
4          -1.048553 -1.420018 -1.706270  1.950775 -0.509652 -0.438074   
...              ...       ...       ...       ...       ...       ...   
9995       -0.030453  1.082681 -0.530580  0.602367 -2.138291 -0.056632   
9996        2.520830 -0.124884  0.018068 -0.998623 -1.262734 -0.565107   
9997       -0.545705  1.696766 -0.508186  0.003621 -1.018515  0.251562   
9998       -0.367787 -0.568358 -0.721248  0.198280 -0.217422  0.914757   
9999        0.428571 -0.164999  0.864963 -0.647662  0.720058 -0.337715   

                  X6        X7       

In [3]:

data = pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([Test, yval], axis=1)
out.index.name = 'InstanceId'
digennoise['digen8_4426'] = current_noise
if current_noise == 0.0: 
    outstring = './data/DigenStudy/No_Noise/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)
else:
    outstring = './data/DigenStudy/Noisy/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)

X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

train_set = pd.concat([X_train, y_train], axis =1)
train_set.index.name = 'InstanceId'
tempstring = './data/testfolder/' + 'digen8_4426' +'_train.csv'
train_set.to_csv(tempstring)
cleanrun = OptimizeClean(dataset_name=['digen8_4426'], n_trials=100, data_path='./data/testfolder')
cleanrun.run()
optimal = cleanrun.params
current_auc = cleanrun.goal
os.remove(tempstring)
tempstring = './data/testfolder/' + 'digen8_4426' +'_test.csv'
test_set = pd.concat([X_test, y_test], axis =1)
test_set.index.name = 'InstanceId'
test_set.to_csv(tempstring)
try:
    most_recent_run = AutoRunner(**optimal, dataset_names='digen8_4426',data_path='./data/testfolder',
                                    gen_report=False, clean=False, class_label='target', 
                                    instance_label='InstanceID', ml_algorithms=["NB", "LR", "DT", "EN"], 
                                    exclude=["GB", "RF","ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
    output_csv = most_recent_run.run(run_para=False)
    performance = pd.read_csv(output_csv)
    summary_chart = performance
    goal = performance["ROC AUC"].max()
    best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
    png_out = output_csv.removesuffix('Summary_performance_mean.csv')
    png_out = png_out + 'Summary_ROC.png'
    final_model_comparison = plt.show(png_out)
except:
    print('EXCEPTION')
os.remove(tempstring)
print(current_auc-goal)


[I 2023-07-03 17:04:53,325] A new study created in memory with name: no-name-7a143b81-c64d-4de7-bb4c-e1c6c338da32
  0%|          | 0/100 [00:00<?, ?it/s]INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426_train
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 9200
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       4755
1      0       4445
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Fe

digen8_4426_train digen8_4426
metadata.pickle digen8_4426
logs digen8_4426
jobs digen8_4426
algInfo.pickle digen8_4426
jobsCompleted digen8_4426
EXCEPTION


INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
27              X2                 X7     0.025504
5               X0                 X5    -0.024838
18              X1                 X8    -0.022384
23              X2                 X3     0.022321
56              X5                 X6    -0.020954
59              X5                 X9    -0.018944
79              X7                 X9    -0.018409
28              X2                 X8    -0.014765
15              X1                 X5    -0.014750
68              X6                 X8    -0.013327
INFO: No Features with correlation higher that parameter
INFO: Running Basic Exploratory Analysis...
INFO: Processed Data Counts

digen8_4426_train digen8_4426
metadata.pickle digen8_4426
logs digen8_4426
jobs digen8_4426
algInfo.pickle digen8_4426
jobsCompleted digen8_4426
EXCEPTION


INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
27              X2                 X7     0.025504
5               X0                 X5    -0.024838
18              X1                 X8    -0.022384
23              X2                 X3     0.022321
56              X5                 X6    -0.020954
59              X5                 X9    -0.018944
79              X7                 X9    -0.018409
28              X2                 X8    -0.014765
15              X1                 X5    -0.014750
68              X6                 X8    -0.013327
INFO: No Features with correlation higher that parameter
INFO: Running Basic Exploratory Analysis...
INFO: Processed Data Counts

...

INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
27              X2                 X7     0.025504
5               X0                 X5    -0.024838
18              X1                 X8    -0.022384
23              X2                 X3     0.022321
56              X5                 X6    -0.020954
59              X5                 X9    -0.018944
79              X7                 X9    -0.018409
28              X2                 X8    -0.014765
15              X1                 X5    -0.014750
68              X6                 X8    -0.013327
INFO: No Features with correlation higher that parameter
INFO: Running Basic Exploratory Analysis...
INFO: Processed Data Counts

digen8_4426_train digen8_4426
metadata.pickle digen8_4426
logs digen8_4426
jobs digen8_4426
algInfo.pickle digen8_4426
jobsCompleted digen8_4426
EXCEPTION


INFO: Initial Data Counts: ----------------
INFO: Instance Count = 9200
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       4755
1      0       4445
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
27              X2                 X7     0.025504
5               X0                 X5    -0.024838
18              X1                 X8    -0.022384
23              X2                 X3     0.022321
56              X5                 X6    -0.020954
59              X5                 X9    

[W 2023-07-03 21:40:26,915] Trial 17 failed with parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth': 15, 'min_samples_split': 30, 'min_samples_leaf': 9, 'max_features': None, 'class_weight': None, 'random_state': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/Users/gabrielketron/AutoTest/STREAMLINE/streamline/modeling/basemodel.py", line 104, in <lambda>
    self.study.optimize(lambda trial: self.objective(trial), n_trials=n_trails, timeout=timeout,
  File "/Users/gabrielketron/AutoTest/STREAMLINE/streamline/models/decision_tree.py", line 35, in objective
    mean_cv_score = self.hyper_eval()
  File "/Users/gabrielketron/AutoTest/STREAMLINE/streamline/modeling/basemodel.py", line 137, in hyper_eval
    mean_cv_score = cross_val_score(model, self.x_train,

INFO: Initial Data Counts: ----------------
INFO: Instance Count = 9200
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       4755
1      0       4445
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
27              X2                 X7     0.025504
5               X0                 X5    -0.024838
18              X1                 X8    -0.022384
23              X2                 X3     0.022321
56              X5                 X6    -0.020954
59              X5                 X9    

categorical cut = 5
sig cutoff = 0.0198
feature missing = 0.7
cleaning miss = 0.6
correlation removal = 0.9
partition method: random
n_splits = 9

424 min runtime for 1 dataset with 100 trials. 
= 7 hours.

In [ ]:
print(cleanrun.param)

{'categorical_cutoff': 10, 'sig_cutoff': 0.04541006026928159, 'featureeng_missingness': 0.8, 'cleaning_missingness': 0.8, 'correlation_removal_threshold': 1.0, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 7}


In [ ]:
# autotest = AutoRunner(dataset_names=['digen8_4426'], clean=False, class_label='y', instance_label='InstanceId')

In [ ]:
#output_csv = autotest.run()

In [ ]:
#print(output_csv)

#performance = pd.read_csv(output_csv)
#print(performance)
#print(performance['ROC AUC'].max())
#print(performance.loc[performance['ROC AUC'].idxmax()][0])